#### LeetCode - SQL - #550

Refer [550. Game Play Analysis IV](https://leetcode.com/problems/game-play-analysis-iv/description/)

Write a solution to report the **fraction** of players that logged in again on the day after the day they first logged in, **rounded to 2 decimal places**. In other words, you need to count the number of players that logged in for at least two consecutive days starting from their first login date, then divide that number by the total number of players.

In [0]:
activity_data = [[1, 2, '2016-03-01', 5], [1, 2, '2016-03-02', 6], [2, 3, '2017-06-25', 1], [3, 1, '2016-03-02', 0], [3, 4, '2018-07-03', 5]]
activity_columns = ['player_id', 'device_id', 'event_date', 'games_played']
activity_schema = {'player_id':'Int64', 'device_id':'Int64', 'event_date':'datetime64[ns]', 'games_played':'Int64'}

In [0]:
import pandas

activity_pandas_dataframe = pandas.DataFrame(data=activity_data, columns=activity_columns).astype(activity_schema)

In [0]:
from pyspark.sql import SparkSession

spark_context = SparkSession.builder.appName("LeetCode SQL").getOrCreate()

# Spark DataFrame using Pandas DataFrame
activity_dataframe = spark_context.createDataFrame(activity_pandas_dataframe)
activity_dataframe.printSchema()
activity_dataframe.show(5)

root
 |-- player_id: long (nullable = true)
 |-- device_id: long (nullable = true)
 |-- event_date: timestamp (nullable = true)
 |-- games_played: long (nullable = true)

+---------+---------+-------------------+------------+
|player_id|device_id|         event_date|games_played|
+---------+---------+-------------------+------------+
|        1|        2|2016-03-01 00:00:00|           5|
|        1|        2|2016-03-02 00:00:00|           6|
|        2|        3|2017-06-25 00:00:00|           1|
|        3|        1|2016-03-02 00:00:00|           0|
|        3|        4|2018-07-03 00:00:00|           5|
+---------+---------+-------------------+------------+



In [0]:
from pyspark.sql.functions import to_date, min

activity_first_login_dataframe \
    = activity_dataframe \
        .groupBy(activity_dataframe.player_id) \
        .agg( \
            min(to_date(activity_dataframe.event_date)).alias("first_login") \
        )
activity_first_login_dataframe.show(5)

+---------+-----------+
|player_id|first_login|
+---------+-----------+
|        1| 2016-03-01|
|        2| 2017-06-25|
|        3| 2016-03-02|
+---------+-----------+



In [0]:
from pyspark.sql.functions import col, date_add, to_date, count, round

activity_summary_dataframe \
    = activity_first_login_dataframe.alias("first") \
        .join(activity_dataframe.alias("second"), (col("first.player_id") == col("second.player_id")) & (date_add(col("first.first_login"), 1) == to_date(col("second.event_date"))), "left") \
        .select(col("first.player_id").alias("player_id"), col("first.first_login").alias("first_login"), to_date(col("second.event_date")).alias("second_login"))

activity_summary_dataframe.select(round((count(activity_summary_dataframe.second_login) / count("*")), 2).alias("fraction")).show()

+--------+
|fraction|
+--------+
|    0.33|
+--------+

